In [1]:
import numpy as np

# !pip install -e .
from nodal_knot import NodalKnot, plot_3D_and_2D_projections

In [2]:
def k_to_zw(kx, ky, kz):
    """ F: 3D Brillouin zone -> C^2 """

    z_real = np.cos(2*kz) + 0.5
    z_imag = np.cos(kx) + np.cos(ky) + np.cos(kz) - 2.0
    z = z_real + 1j*z_imag
    
    w_real = np.sin(kx)
    w_imag = np.sin(ky)
    w = w_real + 1j*w_imag

    return z, w

def zw_to_c_hopf(z, w):
    """ f: C^2 -> C (Hopf Link) """
    return np.power(z, 2) - np.power(w, 2)

def zw_to_c_trefoil(z, w):
    """ f: C^2 -> C (Trefoil Knot) """
    return np.power(z, 2) - np.power(w, 3)

def zw_to_c_figure8(z, w):
    """ f: C^2 -> C (Figure-8 Knot) """
    return np.power(z, 3) - np.power(w, 2)*z

hopf = NodalKnot(k_to_zw, zw_to_c_hopf)
trefoil = NodalKnot(k_to_zw, zw_to_c_trefoil)
figure8 = NodalKnot(k_to_zw, zw_to_c_figure8)

In [3]:
hopf.generate_region(pts_per_dim=300);

In [4]:
hopf_points = hopf.knot_points(thickness=0., epsilon=0.03)
hopf_figure = hopf.plot_3D(hopf_points)
hopf_figure.show()